# Tomorrow.io Weather Data Analysis

This notebook analyzes weather data collected from the Tomorrow.io API for 10 geolocations.

## Setup

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Database connection
from sqlalchemy import create_engine

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Database connection string
db_url = f"postgresql://{os.getenv('PGUSER', 'postgres')}:{os.getenv('PGPASSWORD', 'postgres')}@{os.getenv('PGHOST', 'localhost')}:{os.getenv('PGPORT', '5432')}/{os.getenv('PGDATABASE', 'tomorrow')}"
engine = create_engine(db_url)

print(f"Connected to database: {os.getenv('PGDATABASE', 'tomorrow')}")

## 1. Data Overview

In [ ]:
# Load locations
locations_query = """
SELECT id, lat, lon, name
FROM locations
WHERE is_active = TRUE
ORDER BY id
"""

locations_df = pd.read_sql(locations_query, engine)
print(f"Active Locations: {len(locations_df)}")
locations_df

In [ ]:
# Load recent weather data
weather_query = """
SELECT 
    w.*,
    l.name as location_name,
    l.lat,
    l.lon
FROM weather_data w
JOIN locations l ON w.location_id = l.id
WHERE w.data_granularity = 'hourly'
ORDER BY w.timestamp DESC
LIMIT 1000
"""

weather_df = pd.read_sql(weather_query, engine)
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])

print(f"Total records loaded: {len(weather_df)}")
print(f"\nDate range: {weather_df['timestamp'].min()} to {weather_df['timestamp'].max()}")
print(f"\nLocations with data: {weather_df['location_name'].nunique()}")

weather_df.head()

## 2. Assignment Query 1: Latest Temperature and Wind Speed by Location

In [ ]:
latest_query = """
SELECT DISTINCT ON (l.id)
    l.id as location_id,
    l.name as location_name,
    l.lat,
    l.lon,
    w.timestamp,
    w.temperature,
    w.wind_speed,
    w.humidity,
    w.weather_code
FROM locations l
LEFT JOIN weather_data w ON l.id = w.location_id
WHERE l.is_active = TRUE
ORDER BY l.id, w.timestamp DESC NULLS LAST
"""

latest_df = pd.read_sql(latest_query, engine)
latest_df

In [ ]:
# Visualize latest temperatures
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Temperature by location
latest_df.dropna(subset=['temperature']).plot(
    x='location_name', y='temperature', kind='bar', ax=ax1, color='coral'
)
ax1.set_title('Latest Temperature by Location')
ax1.set_ylabel('Temperature (°C)')
ax1.tick_params(axis='x', rotation=45)

# Wind speed by location
latest_df.dropna(subset=['wind_speed']).plot(
    x='location_name', y='wind_speed', kind='bar', ax=ax2, color='skyblue'
)
ax2.set_title('Latest Wind Speed by Location')
ax2.set_ylabel('Wind Speed (m/s)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 3. Assignment Query 2: Hourly Time Series Analysis

In [ ]:
# Select a location for time series analysis
location_id = 1  # Change to analyze different locations

# Get time series data (last 7 days to next 5 days)
start_time = datetime.now() - timedelta(days=7)
end_time = datetime.now() + timedelta(days=5)

time_series_query = """
SELECT 
    timestamp,
    temperature,
    wind_speed,
    humidity,
    pressure_sea_level
FROM weather_data
WHERE location_id = %(location_id)s
  AND data_granularity = 'hourly'
  AND timestamp BETWEEN %(start)s AND %(end)s
ORDER BY timestamp
"""

time_series_df = pd.read_sql(
    time_series_query, 
    engine, 
    params={'location_id': location_id, 'start': start_time, 'end': end_time}
)
time_series_df['timestamp'] = pd.to_datetime(time_series_df['timestamp'])

location_name = locations_df[locations_df['id'] == location_id]['name'].iloc[0]
print(f"Time series for: {location_name}")
print(f"Records: {len(time_series_df)}")
time_series_df.head()

In [ ]:
# Plot time series
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Temperature
axes[0, 0].plot(time_series_df['timestamp'], time_series_df['temperature'], color='coral')
axes[0, 0].set_title('Temperature Over Time')
axes[0, 0].set_ylabel('Temperature (°C)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Wind Speed
axes[0, 1].plot(time_series_df['timestamp'], time_series_df['wind_speed'], color='skyblue')
axes[0, 1].set_title('Wind Speed Over Time')
axes[0, 1].set_ylabel('Wind Speed (m/s)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Humidity
axes[1, 0].plot(time_series_df['timestamp'], time_series_df['humidity'], color='lightgreen')
axes[1, 0].set_title('Humidity Over Time')
axes[1, 0].set_ylabel('Humidity (%)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Pressure
axes[1, 1].plot(time_series_df['timestamp'], time_series_df['pressure_sea_level'], color='plum')
axes[1, 1].set_title('Sea Level Pressure Over Time')
axes[1, 1].set_ylabel('Pressure (hPa)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.suptitle(f'Weather Time Series: {location_name}', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Comparative Analysis Across Locations

In [ ]:
# Temperature comparison across all locations
temp_comparison_query = """
SELECT 
    l.name as location_name,
    w.timestamp,
    w.temperature
FROM weather_data w
JOIN locations l ON w.location_id = l.id
WHERE w.data_granularity = 'hourly'
  AND w.timestamp >= NOW() - INTERVAL '24 hours'
ORDER BY w.timestamp
"""

temp_df = pd.read_sql(temp_comparison_query, engine)
temp_df['timestamp'] = pd.to_datetime(temp_df['timestamp'])

# Pivot for easier plotting
temp_pivot = temp_df.pivot(index='timestamp', columns='location_name', values='temperature')

# Plot
plt.figure(figsize=(15, 7))
temp_pivot.plot(ax=plt.gca(), alpha=0.8)
plt.title('Temperature Comparison Across Locations (Last 24 Hours)')
plt.ylabel('Temperature (°C)')
plt.xlabel('Time')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 5. Data Quality Check

In [ ]:
# Check data availability
availability_query = """
SELECT 
    l.name as location_name,
    COUNT(*) as record_count,
    MIN(w.timestamp) as earliest,
    MAX(w.timestamp) as latest
FROM weather_data w
JOIN locations l ON w.location_id = l.id
WHERE w.data_granularity = 'hourly'
GROUP BY l.id, l.name
ORDER BY record_count DESC
"""

availability_df = pd.read_sql(availability_query, engine)
availability_df

In [ ]:
# Visualize data availability
plt.figure(figsize=(12, 6))
availability_df.set_index('location_name')['record_count'].plot(kind='bar', color='steelblue')
plt.title('Data Availability by Location')
plt.ylabel('Number of Records')
plt.xlabel('Location')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 6. Statistical Summary

In [ ]:
# Summary statistics by location
stats_query = """
SELECT 
    l.name as location_name,
    AVG(w.temperature) as avg_temp,
    MIN(w.temperature) as min_temp,
    MAX(w.temperature) as max_temp,
    AVG(w.wind_speed) as avg_wind,
    AVG(w.humidity) as avg_humidity
FROM weather_data w
JOIN locations l ON w.location_id = l.id
WHERE w.data_granularity = 'hourly'
GROUP BY l.id, l.name
ORDER BY avg_temp DESC
"""

stats_df = pd.read_sql(stats_query, engine)
stats_df

In [ ]:
# Heatmap of weather metrics
metrics = stats_df.set_index('location_name')[['avg_temp', 'avg_wind', 'avg_humidity']]

plt.figure(figsize=(10, 6))
sns.heatmap(metrics, annot=True, cmap='YlOrRd', fmt='.1f')
plt.title('Average Weather Metrics by Location')
plt.ylabel('Location')
plt.xlabel('Metric')
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. **Assignment Query 1**: Latest temperature and wind speed for each of the 10 geolocations
2. **Assignment Query 2**: Hourly time series analysis showing weather patterns from -1 day to +5 days
3. **Comparative Analysis**: Temperature variations across different locations
4. **Data Quality**: Record counts and availability by location
5. **Statistics**: Average, min, max values for key weather metrics